# Synonymous mutation spectrum
Analyze the synonymous mutation spectrum.

Get input variables from [papermill](https://papermill.readthedocs.io/) parameterization (note next cell is tagged as `parameters`).
So when this notebook is run via `papermill`, those values will be replaced with whatever is in pipeline:

In [1]:
mutation_counts_csv = "../results/mutation_counts/aggregated.csv"
ref_and_founder_nts_csv = "../results/coding_nts/coding_nts.csv"

synonymous_spectra_min_counts = 10000

subset_order = ["all", "USA", "England"]

Import Python modules:

In [2]:
import itertools
import os

import altair as alt

import numpy

import scipy

import pandas as pd

import sklearn.decomposition

Read the mutation counts and assign mutation types:

In [3]:
mutation_counts = pd.read_csv(mutation_counts_csv).assign(
    mut_type=lambda x: x["nt_mutation"].map(lambda m: f"{m[0]}to{m[-1]}")
)

For each clade plot the top mutations as a fraction of all mutations in that clade, just using the "all" subset.

You can mouseover points to highlight mutations (which will highlight all mutations at that site on all facets), and click the legend to show/hide excluded or non-excluded mutations.

This plot is useful to look at to identifier apparent outlier sites with aberrantly high mutation counts that can then be specified for exclusion (note those specifications are done in the pipeline `config.yaml` file, and also all reversions from clade founder to reference may be excluded):

In [4]:
top_n = 100  # plot this many per clade

mutation_freqs = (
    mutation_counts
    .query("subset == 'all'")
    .sort_values(["clade", "count"], ascending=False)
    .groupby("clade")
    .head(n=top_n)
    .assign(
        freq=lambda x: x["count"] / x.groupby("clade")["count"].transform("sum"),
        rank=lambda x: x.groupby("clade")["freq"].rank(ascending=False, method="first"),
        exclude=lambda x: x["exclude"].map({True: "yes", False: "no"}),
    )
)

select_exclude = alt.selection_multi(
    fields=["exclude"], bind="legend", init=[{"exclude": "yes"}, {"exclude": "no"}],
)

select_site = alt.selection_single(
    fields=["nt_site"], on="mouseover", empty="none",
)

mutation_freqs_chart = (
    alt.Chart(mutation_freqs)
    .encode(
        x="rank",
        y="freq",
        strokeWidth=alt.condition(select_site, alt.value(2), alt.value(0)),
        color=alt.Color("exclude", scale=alt.Scale(domain=["yes", "no"])),
        shape=alt.Shape("synonymous"),
        size=alt.condition(select_site, alt.value(50), alt.value(25)),
        tooltip=["nt_site", "nt_mutation", "count", "freq"],
    )
    .mark_point(filled=True, stroke="black")
    .properties(width=200, height=100)
    .facet("clade", columns=4)
    .add_selection(select_exclude, select_site)
    .transform_filter(select_exclude)
)

mutation_freqs_chart

alt.FacetChart(...)

Tally mutation type counts among **only synonymous** mutations for each clade and subset, also removing any mutations specified for exclusion:

In [5]:
mut_type_counts = (
    mutation_counts
    .query("synonymous")
    .query("not exclude")
    .groupby(["clade", "subset", "mut_type"], as_index=False)
    .aggregate({"count": "sum"})
)

Now also repeat these mutation type counts tally, but any mutations in the top 10 most frequent observed mutation for any clade, not doing any subsetting (just taking subset "all"):

In [6]:
exclude_top_n = 5  # exclude mutations in this top rank for any clade

mut_type_counts_exclude_top = (
    mutation_counts
    .query("synonymous")
    .query("not exclude")
    .query("subset == 'all'")
    .assign(
        clade_rank=lambda x: x.groupby("clade")["count"].rank(ascending=False, method="min"),
        highest_rank=lambda x: x.groupby("nt_mutation")["clade_rank"].transform("min"),
    )
    .query("highest_rank > @exclude_top_n")
    .groupby(["clade", "mut_type"], as_index=False)
    .aggregate({"count": "sum"})
)

Plot total mutation counts for each clade and subset on a log scale.
Also draw a line at our minimum cutoff: we only keep subsets above this cutoff:

In [7]:
clade_counts = (
    mut_type_counts
    .groupby(["clade", "subset"], as_index=False)
    .aggregate({"count": "sum"})
)

clade_counts_chart = (
    alt.Chart(clade_counts)
    .encode(
        x="clade",
        y=alt.Y("count", title="total mutations", scale=alt.Scale(type="log")),
        tooltip=["clade", "subset", "count"],
        color="subset",
    )
    .mark_circle(size=50, opacity=0.7)
    .properties(width=alt.Step(18), height=175)
)

# draw cutoff line
cutoff = (
    alt.Chart(pd.DataFrame({"y": [synonymous_spectra_min_counts]}))
    .encode(y="y")
    .mark_rule(strokeDash=[2, 2])
)

(clade_counts_chart + cutoff).configure_axis(grid=False)

alt.LayerChart(...)

For genome partitioning, we subdivide the genome into halves based on the first and last site with an observed mutation:

In [8]:
n_partitions = 2

min_site = mutation_counts["nt_site"].min()
max_site = mutation_counts["nt_site"].max() + 1
partition_bounds = numpy.linspace(min_site, max_site, n_partitions + 1)

def assign_partition(r):
    """Assign nucleotide mutation to its partition."""
    for i in range(1, n_partitions + 1):
        if partition_bounds[i - 1] <= r < partition_bounds[i]:
            return f"partition {i}"

mutation_counts = (
    mutation_counts
    .assign(partition=lambda x: x["nt_site"].map(assign_partition))
)

If we choose to normalize nucleotide composition across compartments, we determine the parental nucleotide count in each sequence for each partition relative to the unpartitioned genome, and calculate the enrichment of each base in each partition:

In [9]:
ref_and_founder_nts = pd.read_csv(ref_and_founder_nts_csv)

nt_composition = (
    pd.concat(
        [
            ref_and_founder_nts.assign(partition="all"),
            ref_and_founder_nts.assign(
                partition=lambda x: x["site"].map(assign_partition),
            ),
        ]
    )
    .groupby(["clade", "partition", "nt"], as_index=False)
    .aggregate(count=pd.NamedAgg("site", "nunique"))
    .assign(
        frac=lambda x: (
            x["count"] / x.groupby(["clade", "partition"])["count"].transform("sum")
        ),
    )
)

partition_enrichment = (
    nt_composition.query("partition != 'all'")
    .merge(
        nt_composition.query("partition == 'all'")[["clade", "nt", "frac"]].rename(
            columns={"frac": "frac_all"}
        ),
        validate="many_to_one",
    )
    .assign(enrichment=lambda x: x["frac"] / x["frac_all"])
)

Get PCA of mutation spectrum, using only filtered synonymous mutation counts for non-excluded mutations for clades/subsets/partitions with adequate counts.

We do the PCA on four different ways of partitioning the data:

 1. Just looking at the "all" subset for each clade across entire genome.
 2. Looking at the "all" subset for each clade across entire genome, but excluding any mutation that is among the top ranked most counts for any clade.
 3. Looking at all subsets for each clade across entire genome.
 4. Looking at the "all" subset along partitions of the genome. 
 
In the plots below, you can mouseover the points for details and click on clades in legends (shift click for multiple clades) to highlight just points for the selected clade(s).
You can also use the scroll bar to only show points with at least the indicated number of total synonymous mutation counts (after filtering):

In [21]:
for title, subsets, partition, exclude_top in [
#    ("all samples, whole genome", ["all"], False, False),
#    ("all samples, whole genome, +/- top mutations", ["all"], False, True),
#    ("by region, whole genome", subset_order, False, False),
    ("all samples, partitioned genome", ["all"], True, False),
]:
    
    filtered_mutation_counts = (
        mutation_counts
        .query("synonymous")
        .query("not exclude")
        .query("subset in @subsets")
    )
    
    if partition:
        filtered_mutation_counts = pd.concat(
            [
                filtered_mutation_counts.assign(partition="all"),
                filtered_mutation_counts,
            ]
        )
    else:
        filtered_mutation_counts = filtered_mutation_counts.assign(partition="all")
        
    mut_type_counts = (
        filtered_mutation_counts
        .groupby(["clade", "subset", "partition", "mut_type"], as_index=False)
        .aggregate({"count": "sum"})
    )
    
    if exclude_top:
        assert all(mut_type_counts["partition"] == "all")
        assert all(mut_type_counts["subset"] == "all")
        mut_type_counts = pd.concat(
            [
                mut_type_counts.assign(excluded="no"),
                mut_type_counts_exclude_top.assign(
                    partition="all", subset="all", excluded="yes",
                ),
            ]
        )
    else:
        mut_type_counts = mut_type_counts.assign(excluded="no")
   
    mut_type_freqs = (
        mut_type_counts
        .assign(
            total_count=lambda x: (
                x.groupby(["clade", "subset", "partition", "excluded"])
                ["count"].transform("sum")
            ),
            freq=lambda x: x["count"] / x["total_count"],
        )
        .query("total_count >= @synonymous_spectra_min_counts")
    )
    
    if not partition:
        # normalize frequencies by partition enrichment
        mut_type_freqs = (
            mut_type_freqs
            .assign(nt=lambda x: x["mut_type"].str[0])
            .merge(
                partition_enrichment[["clade", "partition", "nt", "enrichment"]],
                validate="many_to_one",
                how="left",
            )
            .assign(
                enrichment=lambda x: x["enrichment"].fillna(1),
                freq_non_norm=lambda x: x["freq"] / x["enrichment"],
                freq=lambda x: (
                    x["freq_non_norm"] /
                    x.groupby(["clade", "partition"])["freq_non_norm"].transform("sum")
                ),
            )
        )
        
        display(
            mut_type_freqs
            .groupby(["clade", "partition"])
            .aggregate({"freq": "sum", "freq_non_norm": "sum"})
        )
        
#        display(mut_type_freqs)
#        display(partition_enrichment)
#        raise RuntimeError
        
    mut_type_freqs = (
        mut_type_freqs
        .pivot_table(
            index=["clade", "subset", "partition", "excluded", "total_count"],
            values="freq",
            columns="mut_type",
            fill_value=0,
        )
    )
    
    pca = sklearn.decomposition.PCA(n_components=2)
    pca_coords = pca.fit_transform(mut_type_freqs.values)
    assert len(pca_coords) == len(mut_type_freqs)

    mut_type_freqs_pca_coords = (
        mut_type_freqs
        .reset_index()
        .assign(
            principal_component_1=pca_coords[:, 0],
            principal_component_2=pca_coords[:, 1],
            log10_total_count=lambda x: numpy.log(x["total_count"]) / numpy.log(10),
        )
    )
    
    # percent variance explained by each component
    pca_var = 100 * pca.explained_variance_ratio_
    
    total_count_selection = alt.selection_single(
        fields=["log10_total_count"],
        init={"log10_total_count": 4},
        bind=alt.binding_range(
            name="minimum log10 total counts",
            min=int(mut_type_freqs_pca_coords["log10_total_count"].min()),
            max=mut_type_freqs_pca_coords["log10_total_count"].max(),
        )
    )
    
    clade_selection = alt.selection_multi(fields=["clade"], bind="legend")

    tooltip = ["clade", "total_count"]
    
    plot_size = 300  # scaled by component variance explained
    
    pca_chart = (
        alt.Chart(mut_type_freqs_pca_coords)
        .encode(
            y=alt.Y(
                "principal_component_1",
                title=f"PC1 ({pca_var[0]:.0f}% variance)",
                scale=alt.Scale(nice=False, padding=10),
                axis=alt.Axis(labels=False, ticks=False),
            ),
            x=alt.X(
                "principal_component_2",
                title=f"PC2 ({pca_var[1]:.0f}% variance)",
                scale=alt.Scale(nice=False, padding=10),
                axis=alt.Axis(labels=False, ticks=False),
            ),
            color=alt.Color("clade", scale=alt.Scale(scheme="viridis")),
            strokeWidth=alt.condition(clade_selection, alt.value(1.5), alt.value(0)),
            opacity=alt.condition(clade_selection, alt.value(0.9), alt.value(0.45)),
            size=alt.condition(clade_selection, alt.value(65), alt.value(45)),
        )
        .mark_point(filled=True, stroke="black")
        .add_selection(total_count_selection, clade_selection)
        .transform_filter(
            total_count_selection.log10_total_count <= alt.datum.log10_total_count
        )
        .configure_axis(grid=False)
        .configure_legend(columns=2)
        .properties(
            height=plot_size, width=plot_size * pca_var[1] / pca_var[0],
            title=title,
        )
    )
    
    if len(subsets) > 1:
        pca_chart = pca_chart.encode(shape=alt.Shape("subset", sort=subset_order))
        tooltip.append("subset")
        
    if partition:
        pca_chart = pca_chart.encode(shape="partition")
        tooltip.append("partition")
        
    if exclude_top:
        pca_chart = pca_chart.encode(
            shape=alt.Shape(
                "excluded", title=f"excluding top {exclude_top_n} per clade",
            ),
        )
        tooltip.append(
            alt.Tooltip("excluded", title=f"excluding top {exclude_top_n} per clade")
        )
                       
    pca_chart = pca_chart.encode(tooltip=tooltip)

    display(pca_chart)
    print("\n\n")

alt.Chart(...)

Compute statistical significance of differences between clades.
We just do this on "all" sequences for a clade, not partitioning the genomes:

In [ ]:
all_mut_type_counts = (
    mut_type_counts.query("subset == 'all'")
    .drop(columns="subset")
    .assign(total_count=lambda x: x.groupby("clade")["count"].transform("sum"))
    .query("total_count >= @synonymous_spectra_min_counts")
    .drop(columns="total_count")
)

wide_all_mut_type_counts = all_mut_type_counts.pivot_table(
    index="mut_type",
    columns="clade",
    values="count",
    fill_value=0,
)

Now run chi2 test.
Also, Bonferroni correct the P-values (this is conservative, but is fine as these P-values are so tiny):

In [ ]:
min_p = 1e-20  # plot P-values less than this as this

records = []
for clade1, clade2 in itertools.combinations(wide_all_mut_type_counts.columns, 2):
    chi2, p, dof, _ = scipy.stats.chi2_contingency(
        wide_all_mut_type_counts[[clade1, clade2]]
    )
    records.append((clade1, clade2, p, chi2))
    
chi2_stats = (
    pd.DataFrame(records, columns=["clade_1", "clade_2", "p", "chi2"])
    .assign(
        p=lambda x: x["p"].clip(lower=min_p),
        bonferroni_p=lambda x: (x["p"] * len(x)).clip(upper=1),
    )
)

Plot the Bonferroni corrected P-values.
Note since counts are very large, many comparisons will be highly significant:

In [ ]:
p_chart = (
    alt.Chart(chi2_stats)
    .encode(
        x=alt.X("clade_1", title=None),
        y=alt.Y("clade_2", title=None),
        fill=alt.Fill(
            "bonferroni_p",
            title="Bonferroni corrected P-value",
            scale=alt.Scale(type="log", scheme="yelloworangered", reverse=True),
            legend=alt.Legend(orient="top"),
        ),
        tooltip=[
            "clade_1",
            "clade_2",
            alt.Tooltip("p", format=".2g"),
            alt.Tooltip("bonferroni_p", format=".2g"),
            alt.Tooltip("chi2", format=".2g"),
        ],
    )
    .mark_rect(stroke="black")
    .properties(width=alt.Step(14), height=alt.Step(14))
)

p_chart